In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

jdbc_url = (
    "jdbc:sqlserver://sqlsrv-fr-risk-neu.database.windows.net:1433;"
    "database=sqldb-fr-risk;encrypt=true;hostNameInCertificate=*.database.windows.net;"
    "loginTimeout=30;"
)

props = {
    "user": "CloudSA15903b81",
    "password": "Fraud@123",
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

In [0]:
#Load all Silver tables
accounts     = spark.read.jdbc(jdbc_url, "silver.accounts",     properties=props)
transactions = spark.read.jdbc(jdbc_url, "silver.transactions", properties=props)
merchants    = spark.read.jdbc(jdbc_url, "silver.merchants",    properties=props)
devices      = spark.read.jdbc(jdbc_url, "silver.devices",      properties=props)
login_events = spark.read.jdbc(jdbc_url, "silver.login_events", properties=props)
chargebacks  = spark.read.jdbc(jdbc_url, "silver.chargebacks",  properties=props)
ip_rep       = spark.read.jdbc(jdbc_url, "silver.ip_reputation",properties=props)


In [0]:
login_events = spark.read.jdbc(jdbc_url, "silver.login_events", properties=props)
ip_rep       = spark.read.jdbc(jdbc_url, "silver.ip_reputation", properties=props)

overlap = (
    login_events.select("ip").distinct()
    .join(ip_rep.select("ip").distinct(), "ip", "inner")
    .count()
)
print("Distinct overlapping IPs:", overlap)


Distinct overlapping IPs: 16088


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
#These are being done later after finding issue in PBI
# Load Silver tables (already done above)
# accounts, transactions, merchants, devices, login_events, chargebacks, ip_rep

# 1. Join login events with IP reputation
login_joined = (
    login_events.alias("l")
    .join(
        ip_rep.select("ip", "reputation_score").alias("ip"),
        on="ip",
        how="left"
    )
)

# 2. Aggregate metrics per customer_id
login_risk = (
    login_joined
    .groupBy("customer_id")
    .agg(
        F.count("*").alias("login_count"),
        F.sum(F.when(F.col("success") == False, 1).otherwise(0)).alias("failed_logins"),
        F.sum(F.when((F.col("success") == False) & (F.col("mfa_used") == False), 1).otherwise(0)).alias("failed_no_mfa"),
        F.avg("reputation_score").alias("avg_ip_reputation"),
        F.countDistinct("ip").alias("unique_ips"),
        F.countDistinct("device_id").alias("unique_devices"),
        F.sum(F.when(F.col("device_id").isin(
            devices.filter(F.col("is_rooted") == True).select("device_id").rdd.flatMap(lambda x: x).collect()
        ), 1).otherwise(0)).alias("rooted_device_logins")
    )
)

# Write to Gold
login_risk.write.jdbc(
    url=jdbc_url,
    table="gold.login_risk",
    mode="overwrite",
    properties=props
)


In [0]:
# Join chargebacks with transactions to bring in account_id
cb_txn = (
    chargebacks.alias("cb")
    .join(
        transactions.select("txn_id", "account_id").alias("t"),
        on="txn_id",
        how="left"
    )
)

cb_txn.printSchema()


root
 |-- txn_id: string (nullable = true)
 |-- cb_id: string (nullable = true)
 |-- reason_code: string (nullable = true)
 |-- filed_dt: timestamp (nullable = true)
 |-- won_dispute: boolean (nullable = true)
 |-- loaded_at: timestamp (nullable = true)
 |-- account_id: string (nullable = true)



In [0]:
#2.1 Txn aggregation per account
acct_txn_agg = (
    transactions
    .withColumn("txn_dt_ts", F.col("txn_dt").cast("timestamp"))
    .groupBy("account_id")
    .agg(
        F.count("*").alias("txn_count"),
        F.sum("amount").alias("total_spend"),
        F.avg("amount").alias("avg_txn_amount"),
        F.max("amount").alias("max_txn_amount"),
        F.min("txn_dt_ts").alias("first_txn_dt"),
        F.max("txn_dt_ts").alias("last_txn_dt"),
        F.countDistinct("merchant_id").alias("unique_merchants"),
        F.sum(F.when(F.col("is_international") == 1, 1).otherwise(0)).alias("intl_txn_count"),
        F.sum(F.when(F.col("is_high_amount") == 1, 1).otherwise(0)).alias("high_amt_txn_count"),
        F.sum(F.when(F.col("is_night") == 1, 1).otherwise(0)).alias("night_txn_count"),
    )
)

#2.2 Chargebacks per account using txn_id → account_id
cb_by_acct = (
    cb_txn
    .groupBy("account_id")
    .agg(
        F.count("*").alias("chargeback_count"),
        F.max("filed_dt").alias("last_chargeback_dt"),
        F.sum(F.when(F.col("won_dispute") == 1, 1).otherwise(0)).alias("won_dispute_count")
    )
)


#2.3 Join with accounts master
account_360 = (
    accounts
    .join(acct_txn_agg, "account_id", "left")
    .join(cb_by_acct, "account_id", "left")
)

account_360.write.jdbc(
    url=jdbc_url,
    table="gold.account_360",
    mode="overwrite",
    properties=props
)


In [0]:
#1. Merchant risk profile
#Aggregate transactions per merchant_id
merch_txn_agg = (
    transactions
    .groupBy("merchant_id")
    .agg(
        F.count("*").alias("txn_count"),
        F.sum("amount").alias("total_volume"),
        F.avg("amount").alias("avg_txn_amount"),
        F.stddev("amount").alias("amt_stddev"),
        F.countDistinct("account_id").alias("unique_accounts"),
        F.sum(F.when(F.col("is_fraud") == 1, 1).otherwise(0)).alias("flagged_fraud_count")
    )
)

#Chargebacks per merchant (via txn_id -> merchant_id)
cb_txn_merch = (
    chargebacks.alias("cb")
    .join(
        transactions.select("txn_id", "merchant_id").alias("t"),
        on="txn_id",
        how="left"
    )
)

cb_by_merch = (
    cb_txn_merch
    .groupBy("merchant_id")
    .agg(
        F.count("*").alias("cb_count"),
        F.max("filed_dt").alias("last_cb_dt")
    )
)

#Start from merchants that have transactions, then bring merchant attributes
merchant_risk = (
    merch_txn_agg
    .join(merchants, "merchant_id", "left")     # add MCC, risk_bucket, country_iso, etc.
    .join(cb_by_merch, "merchant_id", "left")   # add cb_count, last_cb_dt
    .withColumn(
        "chargeback_rate",
        F.when(F.col("txn_count") > 0, F.col("cb_count") / F.col("txn_count")).otherwise(F.lit(0.0))
    )
)

#Replace any remaining NULL metrics with 0 (just in case)
merchant_risk = merchant_risk.fillna({
    "txn_count": 0,
    "total_volume": 0.0,
    "avg_txn_amount": 0.0,
    "amt_stddev": 0.0,
    "unique_accounts": 0,
    "flagged_fraud_count": 0,
    "cb_count": 0,
})

#Write to GOLD
merchant_risk.write.jdbc(
    url=jdbc_url,
    table="gold.merchant_risk",
    mode="overwrite",
    properties=props
)

In [0]:
#2. Transaction-level features
w_acct_time = (
    Window
    .partitionBy("account_id")
    .orderBy(F.col("txn_dt").cast("timestamp"))
)

txn_features = (
    transactions
    .withColumn("txn_ts", F.col("txn_dt").cast("timestamp"))
    .withColumn("prev_txn_ts", F.lag("txn_ts").over(w_acct_time))
    .withColumn("prev_amount", F.lag("amount").over(w_acct_time))
    .withColumn("amount_change", F.col("amount") - F.col("prev_amount"))
    .withColumn(
        "mins_since_prev_txn",
        (F.col("txn_ts").cast("long") - F.col("prev_txn_ts").cast("long")) / 60.0
    )
)

txn_features.write.jdbc(
    url=jdbc_url,
    table="gold.transaction_features",
    mode="overwrite",
    properties=props
)

In [0]:
#3. Login risk by customer_id
login_enriched = (
    login_events
    .join(ip_rep.select("ip", "reputation_score"), on="ip", how="left")
    .join(devices.select("device_id", "device_type", "os", "is_rooted"), on="device_id", how="left")
)

login_risk = (
    login_enriched
    .groupBy("customer_id")
    .agg(
        F.count("*").alias("login_count"),
        F.sum(F.when(F.col("success") == 0, 1).otherwise(0)).alias("failed_logins"),
        F.sum(F.when((F.col("success") == 0) & (F.col("mfa_used") != "Y"), 1).otherwise(0)).alias("failed_no_mfa"),
        F.avg("reputation_score").alias("avg_ip_reputation"),
        F.countDistinct("ip").alias("unique_ips"),
        F.countDistinct("device_id").alias("unique_devices"),
        F.sum(F.when(F.col("is_rooted") == 1, 1).otherwise(0)).alias("rooted_device_logins")
    )
)

login_risk.write.jdbc(
    url=jdbc_url,
    table="gold.login_risk",
    mode="overwrite",
    properties=props
)


In [0]:
#4. Fraud labels per transaction
cb_labels = (
    chargebacks
    .select("txn_id")
    .dropDuplicates()
    .withColumn("is_chargeback", F.lit(1))
)

fraud_labels = (
    transactions
    .join(cb_labels, on="txn_id", how="left")
    .withColumn(
        "is_chargeback",
        F.when(F.col("is_chargeback").isNull(), F.lit(0)).otherwise(F.col("is_chargeback"))
    )
)

fraud_labels.write.jdbc(
    url=jdbc_url,
    table="gold.fraud_labels",
    mode="overwrite",
    properties=props
)


In [0]:
#5. Daily account aggregates
daily_account_agg = (
    transactions
    .withColumn("txn_date", F.to_date("txn_dt"))
    .groupBy("account_id", "txn_date")
    .agg(
        F.sum("amount").alias("daily_spend"),
        F.count("*").alias("daily_txn_count"),
        F.avg("amount").alias("daily_avg_amount"),
        F.sum(F.when(F.col("is_fraud") == 1, 1).otherwise(0)).alias("daily_fraud_flags")
    )
)

daily_account_agg.write.jdbc(
    url=jdbc_url,
    table="gold.daily_account_agg",
    mode="overwrite",
    properties=props
)
